## Compute RMSD between two predicted protein structures & Visualize
Ref: https://notebook.community/aloctavodia/SBioA/English/04_Comparing_structures
Ref: https://github.com/aws-solutions-library-samples/aws-batch-arch-for-protein-folding/blob/0a0a62faa0d407f4856e0865df4c8e8ec6d26290/src/batchfold/utils/utils.py#LL102C5-L102C5

In [ ]:
# install pymol https://pymol.org/2/?#download
!conda install -y -c schrodinger pymol-bundle

In [ ]:
!pip install -q py3dmol

In [ ]:
!apt-get update;apt-get install -y libgl1 

In [ ]:
!pip install dm-tree

In [ ]:
!pip install biopython

In [ ]:
import sys, pymol
from pymol import cmd, stored
import py3Dmol
import matplotlib.pyplot as plt

from Bio.PDB import PDBParser, PDBIO
import io
from source_dir import protein
from source_dir import residue_constants

In [ ]:
import numpy as np
import seaborn as sns
import sagemaker
import boto3
client = boto3.client("sagemaker")

In [ ]:
#By default the notebook compares the last 3 successful pipeline executions 
#Provide 3 pipeline executions to override 
pipeline_arns = []

#Below names should correspond to protein file input for last 3 succesful pipeline executions
input_name_1 = "T1090"
input_name_2 = "T1076"
input_name_3 = "T1030"

In [ ]:
past_pipeline_executions = client.list_pipeline_executions(
    PipelineName='ProteinFoldWorkflow',
    MaxResults=25
)

successful_pipeline_executions = []
pipeline_executions_summaries = past_pipeline_executions.get("PipelineExecutionSummaries")
for executionNum in range(len(pipeline_executions_summaries)):
    if pipeline_executions_summaries[executionNum].get("PipelineExecutionStatus") == "Succeeded":
        successful_pipeline_executions.append(past_pipeline_executions.get("PipelineExecutionSummaries")[executionNum])
        
successful_pipeline_executions

In [ ]:
if not pipeline_arns:
    for pipeline_num in range(3):
        pipeline_arns.append(successful_pipeline_executions[pipeline_num].get("PipelineExecutionArn"))
pipeline_arns

In [ ]:
def get_inference_outputs(pipeline_arn):
    client = boto3.client("sagemaker")
    pipeline_output = client.list_pipeline_execution_steps(
        PipelineExecutionArn=pipeline_arn,
        MaxResults=3,
    )
    for pipeline_step in pipeline_output.get("PipelineExecutionSteps"):
        if pipeline_step.get("StepName") == "RunAlphaFold":
            alphafold_job_arn = pipeline_step.get("Metadata").get("TrainingJob").get("Arn")
        elif pipeline_step.get("StepName") == "RunOpenFold":
            openfold_job_arn = pipeline_step.get("Metadata").get("TrainingJob").get("Arn")
    
    alphafold_job_name_index = alphafold_job_arn.find("/")
    openfold_job_name_index = openfold_job_arn.find("/")
    
    alphafold_job_name = alphafold_job_arn[alphafold_job_name_index+1:]
    openfold_job_name = openfold_job_arn[openfold_job_name_index+1:]
    
    alphafold_job_arn = client.describe_training_job(TrainingJobName=alphafold_job_name)
    openfold_job_arn = client.describe_training_job(TrainingJobName=openfold_job_name)
    
    alphafold_output = alphafold_job_arn.get("ModelArtifacts").get("S3ModelArtifacts")
    openfold_output = openfold_job_arn.get("ModelArtifacts").get("S3ModelArtifacts")
    
    return [alphafold_output,openfold_output]

In [ ]:
outputs_1 = get_inference_outputs(pipeline_arns[0])
s3_alphafold_output_1 = outputs_1[0]
s3_openfold_output_1 = outputs_1[1]

outputs_2 = get_inference_outputs(pipeline_arns[1])
s3_alphafold_output_2 = outputs_2[0]
s3_openfold_output_2 = outputs_2[1]

outputs_3 = get_inference_outputs(pipeline_arns[2])
s3_alphafold_output_3 = outputs_3[0]
s3_openfold_output_3 = outputs_3[1]

In [ ]:
sagemaker.s3.S3Downloader.download(s3_alphafold_output_1, local_path=f'output/{input_name_1}/alphafold')
sagemaker.s3.S3Downloader.download(s3_openfold_output_1, local_path=f'output/{input_name_1}/openfold')

sagemaker.s3.S3Downloader.download(s3_alphafold_output_2, local_path=f'output/{input_name_2}/alphafold')
sagemaker.s3.S3Downloader.download(s3_openfold_output_2, local_path=f'output/{input_name_2}/openfold')

sagemaker.s3.S3Downloader.download(s3_alphafold_output_3, local_path=f'output/{input_name_3}/alphafold')
sagemaker.s3.S3Downloader.download(s3_openfold_output_3, local_path=f'output/{input_name_3}/openfold')

In [ ]:
!tar zxfv ./output/{input_name_1}/alphafold/model.tar.gz --directory ./output/{input_name_1}/alphafold/
!tar zxfv ./output/{input_name_1}/openfold/model.tar.gz --directory ./output/{input_name_1}/openfold/

!tar zxfv ./output/{input_name_2}/alphafold/model.tar.gz --directory ./output/{input_name_2}/alphafold/
!tar zxfv ./output/{input_name_2}/openfold/model.tar.gz --directory ./output/{input_name_2}/openfold/

!tar zxfv ./output/{input_name_3}/alphafold/model.tar.gz --directory ./output/{input_name_3}/alphafold/
!tar zxfv ./output/{input_name_3}/openfold/model.tar.gz --directory ./output/{input_name_3}/openfold/

In [ ]:
cmd.load(filename = f'output/{input_name_1}/alphafold/{input_name_1}/ranked_0.pdb', object = f'{input_name_1}_ranked_0')
cmd.load(f'output/{input_name_1}/openfold/predictions/{input_name_1}_model_1_ptm_relaxed.pdb')

cmd.load(filename = f'output/{input_name_2}/alphafold/{input_name_2}/ranked_0.pdb', object = f'{input_name_2}_ranked_0')
cmd.load(f'output/{input_name_2}/openfold/predictions/{input_name_2}_model_1_ptm_relaxed.pdb')

cmd.load(filename = f'output/{input_name_3}/alphafold/{input_name_3}/ranked_0.pdb', object = f'{input_name_3}_ranked_0')
cmd.load(f'output/{input_name_3}/openfold/predictions/{input_name_3}_model_1_ptm_relaxed.pdb')

In [ ]:
cmd.remove('not polymer or hydro')
object0 = cmd.get_names()[0]
object1 = cmd.get_names()[1]

object2 = cmd.get_names()[2]
object3 = cmd.get_names()[3]

object4 = cmd.get_names()[4]
object5 = cmd.get_names()[5]

In [ ]:
object0, object1, object2, object3, object4, object5

## Example of Visualizing Alphafold & Openfold

Visualization code credit: https://notebook.community/aloctavodia/SBioA/English/04_Comparing_structures

In [ ]:
def overwrite_b_factors(pdb_str: str, bfactors: np.ndarray) -> str:
    """Overwrites the B-factors in pdb_str with contents of bfactors array.

    Args:
      pdb_str: An input PDB string.
      bfactors: A numpy array with shape [1, n_residues, 37]. We assume that the
        B-factors are per residue; i.e. that the nonzero entries are identical in
        [0, i, :].

    Returns:
      A new PDB string with the B-factors replaced.
    """
    if bfactors.shape[-1] != residue_constants.atom_type_num:
        raise ValueError(
            f"Invalid final dimension size for bfactors: {bfactors.shape[-1]}."
        )

    parser = PDBParser(QUIET=True)
    handle = io.StringIO(pdb_str)
    structure = parser.get_structure("", handle)

    curr_resid = ("", "", "")
    idx = -1
    for atom in structure.get_atoms():
        atom_resid = atom.parent.get_id()
        if atom_resid != curr_resid:
            idx += 1
            if idx >= bfactors.shape[0]:
                raise ValueError(
                    "Index into bfactors exceeds number of residues. "
                    "B-factors shape: {shape}, idx: {idx}."
                )
        curr_resid = atom_resid
        atom.bfactor = bfactors[idx, residue_constants.atom_order["CA"]]

    new_pdb = io.StringIO()
    pdb_io = PDBIO()
    pdb_io.set_structure(structure)
    pdb_io.save(new_pdb)
    return new_pdb.getvalue()

def plot_banded_pdb(pdb_file, show_sidechains = False, width = 800, height = 600):
    with open(pdb_file) as f:
            best_pdb = f.read()
    target_protein = protein.from_pdb_string(best_pdb)
    plddt_list = target_protein.b_factors[:,0]
    atom_mask = target_protein.atom_mask
    banded_b_factors = []
    for plddt in plddt_list:
        for idx, (min_val, max_val, _) in enumerate(residue_constants.PLDDT_BANDS):
            if plddt >= min_val and plddt <= max_val:
                banded_b_factors.append(idx)
                break

    banded_b_factors = (
            np.array(banded_b_factors)[:, None] * atom_mask
    )

    to_visualize_pdb = overwrite_b_factors(best_pdb, banded_b_factors)
    # Color the structure by per-residue pLDDT
    color_map = {i: bands[2] for i, bands in enumerate(residue_constants.PLDDT_BANDS)}
    view = py3Dmol.view(width, height)
    view.addModelsAsFrames(to_visualize_pdb)
    style = {"cartoon": {"colorscheme": {"prop": "b", "map": color_map}}}
    if show_sidechains:
        style["stick"] = {}
    view.setStyle({"model": -1}, style)
    view.zoomTo()
    view.show()
    return None

def plot_plddt_legend():
    """Plots the legend for pLDDT."""

    thresh = [
        "Very low (pLDDT < 50)",
        "Low (70 > pLDDT > 50)",
        "Confident (90 > pLDDT > 70)",
        "Very high (pLDDT > 90)",
    ]

    colors = [x[2] for x in residue_constants.PLDDT_BANDS]

    plt.figure(figsize=(2, 2))
    for c in colors:
        plt.bar(0, 0, color=c)
    plt.legend(thresh, frameon=False, loc="center", fontsize=20)
    plt.xticks([])
    plt.yticks([])
    ax = plt.gca()
    ax.spines["right"].set_visible(False)
    ax.spines["top"].set_visible(False)
    ax.spines["left"].set_visible(False)
    ax.spines["bottom"].set_visible(False)
    plt.title("Model Confidence", fontsize=20, pad=20)
    return plt

In [ ]:
#AlphaFold
plt=plot_plddt_legend()
plt.show()
plot_banded_pdb(f'output/{input_name_1}/alphafold/{input_name_1}/ranked_0.pdb')

In [ ]:
#OpenFold
plt=plot_plddt_legend()
plt.show()
plot_banded_pdb(f'output/{input_name_1}/openfold/predictions/{input_name_1}_model_1_ptm_relaxed.pdb')

### RMSD (root mean square deviation)
It is the most common metric used to compare two protein structure. It measures the average distance between the atoms of the superimposed proteins.

In [ ]:
def rmsd_cur(mol0, mol1, sel='*'):
    """
    Computes the root mean square deviation from the current
    coordinates of two pairs of equivalent atoms. Does not
    perform a superposition.
    
    Parameters
    ----------
    mol0 : PyMOL object
    mol1 : PyMOL object
    sel  : PyMOL selection, atoms used to compute rmsd.
           e.g. use ca+c+n for the backbone
    """
    model0 = cmd.get_model('%s and name %s' % (mol0, sel))
    model1 = cmd.get_model('%s and name %s'  % (mol1, sel))
    xyz0 = np.array(model0.get_coord_list())
    xyz1 = np.array(model1.get_coord_list())
    
    rmsd = (np.sum((xyz0 - xyz1 )**2)/len(xyz0))**0.5
    return rmsd


In [ ]:
rmsd_cur_one = rmsd_cur(object0, object1, sel='ca') #'ca+c+n'
rmsd_cur_two = rmsd_cur(object2, object3, sel='ca')
rmsd_cur_three = rmsd_cur(object4, object5, sel='ca')
print('%.2f' % rmsd_cur_one,rmsd_cur_two,rmsd_cur_three)

### RMSD with superposition
RMSD can be meaningless if the proteins are not superposed. Superposition two protein structures aligns the structure by rotating and translating one of them respect to the other.

In [ ]:
def rmsd_fit(mol0, mol1, sel='*', fit=True):
    """
    Computes the root mean square deviation from two pairs of
    equivalent atoms after superposition.
    
    Parameters
    ----------
    mol0 : PyMOL object
    mol1 : PyMOL object
    sel  : PyMOL selection. atoms used to compute rmsd.
           e.g. use ca+c+n for the backbone
    fit  : bool. If false computes the rmsd after superposition, but without
           updating the coordinates
           
    """
    xyz0 = np.array(cmd.get_model('%s and name %s' % (mol0, sel)).get_coord_list())
    xyz1 = np.array(cmd.get_model('%s and name %s'  % (mol1, sel)).get_coord_list())
    
    xyz0_all = np.array(cmd.get_model('%s' % mol0).get_coord_list())
    xyz1_all = np.array(cmd.get_model('%s'  % mol1).get_coord_list())
    
    # Translation
    X = xyz0 - xyz0.mean(axis=0)
    Y = xyz1 - xyz1.mean(axis=0)
    # Covariation matrix
    Cov_matrix = np.dot(Y.T, X)
    # Optimal rotation matrix
    U, S, Wt = np.linalg.svd(Cov_matrix)
    # Create Rotation matrix R
    R = np.dot(U, Wt)
    # Ensure a right-handed coordinate system
    if np.linalg.det(R) < 0.:
        S[-1] = -S[-1]
        Wt[-1] *= -1
        R = np.dot(U, Wt) 
    if fit:
        # center the first molecule
        stored.sel0 = list(xyz0_all - xyz0.mean(axis=0))
        # rotate and translate the second molecule
        stored.sel1 = list(np.dot((xyz1_all - xyz1.mean(0)), R))
        #update the changes to the coordinates 
        cmd.alter_state(1, mol0,"(x,y,z)=stored.sel0.pop(0)")
        cmd.alter_state(1, mol1,"(x,y,z)=stored.sel1.pop(0)")

    # We compute the RMSD after superposition by using the matrix S. The advantage is 
    # we do not need to actually do the superposition before computing the RMSD.
    #rmsd = (np.exp(np.log(np.sum(X ** 2) + np.sum(Y ** 2)) - 2.0 * np.log(np.sum(S)))/len(X))**0.5
    rmsd = ((np.sum(X ** 2) + np.sum(Y ** 2) - 2.0 * np.sum(S))/len(X))**0.5
    # scales and translates the window to show a selection
    cmd.zoom()
    return rmsd

In [ ]:
rmsd_fit_one = rmsd_fit(object0, object1, sel='ca', fit=True)
rmsd_fit_two = rmsd_fit(object2, object3, sel='ca', fit=True)
rmsd_fit_three = rmsd_fit(object4, object5, sel='ca', fit=True)
print('%.2f' % rmsd_fit_one,rmsd_fit_two,rmsd_fit_three)

In [ ]:
def tm_score(mol0, mol1, sel='*'): #Check if TM-align use all atoms!
    """
    Compute TM-score between two set of coordinates
    
    Parameters
    ----------
    mol0 : PyMOL object
    mol1 : PyMOL object
    sel  : PyMOL selection, atoms used to compute rmsd.
           e.g. use ca+c+n for the backbone
    """
    xyz0 = np.array(cmd.get_model('%s and name %s' % (mol0, sel)).get_coord_list())
    xyz1 = np.array(cmd.get_model('%s and name %s'  % (mol1, sel)).get_coord_list())
    
    L = len(xyz0)
    # d0 is less than 0.5 for L < 22 
    # and nan for L < 15 (root of a negative number)
    d0 = 1.24 * np.power(L - 15, 1/3) - 1.8
    d0 = max(0.5, d0) 

    # compute the distance for each pair of atoms
    di = np.sum((xyz0 - xyz1) ** 2, 1) # sum along first axis
    return np.sum(1 / (1 + (di / d0) ** 2)) / L

In [ ]:
tmscore_one = tm_score(object0, object1, sel='ca')
tmscore_two = tm_score(object2, object3, sel='ca')
tmscore_three = tm_score(object4, object5, sel='ca')
print('%.4f' % tmscore_one,tmscore_two,tmscore_three)

In [ ]:
!pip install sagemaker-experiments

In [ ]:
%%time
from sagemaker.experiments.run import Run, load_run
import sagemaker
# create an experiment and start a new run
experiment_name='proteinfoldworkflow'
sess=sagemaker.Session()

In [ ]:
metric_type='compare:'
experiment_name = 'proteinfoldworkflow'
with Run(experiment_name=experiment_name, run_name=input_name_1, sagemaker_session=sess) as run:
    run.log_metric(name=metric_type + "rmsd_cur", value=rmsd_cur_one, step=1)
    run.log_metric(name=metric_type + "rmds_fit", value=rmsd_fit_one, step=1)
    run.log_metric(name=metric_type + "tm_score", value=tmscore_one, step=1)

In [ ]:
metric_type='compare:'
experiment_name = 'proteinfoldworkflow'
with load_run(experiment_name=experiment_name, run_name=input_name_2, sagemaker_session=sess) as run:
    run.log_metric(name=metric_type + "rmsd", value=rmsd_fit_two, step=1)
    run.log_metric(name=metric_type + "rmds_fit", value=rmsd_fit_two, step=1)
    run.log_metric(name=metric_type + "tm_score", value=tmscore_two, step=1)

In [ ]:
metric_type='compare:'
experiment_name = 'proteinfoldworkflow'
with load_run(experiment_name=experiment_name, run_name=input_name_3, sagemaker_session=sess) as run:
    run.log_metric(name=metric_type + "rmsd", value=rmsd_three, step=1)
    run.log_metric(name=metric_type + "rmds_fit", value=rmsd_fit_three, step=1)
    run.log_metric(name=metric_type + "tm_score", value=tmscore_three, step=1)